In [1]:
import tensorflow
from tensorflow import keras
from keras import models 
import json
import numpy as np
import pickle
import nltk
from nltk.corpus import stopwords
import random
import spacy

In [2]:
model=models.load_model('ChatBotModel.h5')
words=pickle.load(open('word.pkl','rb'))
intents = json.loads(open('intents.json').read())
classes = pickle.load(open('classes.pkl','rb'))
lemmatizer=nltk.WordNetLemmatizer()
nlp= spacy.load(r".\output\model-best") #load the best model

In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [5]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return np.array(bag)

In [6]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    print(res)
    ERROR_THRESHOLD = 0.1
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [15]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result,tag

In [70]:

def EntityDiseasePrediction(question):
    response1={}
    while 1:  
        doc=nlp(question)
        response1['DISEASE']=""
        for d in doc.ents:
            response1[d.label_]=d.text
        print(response1)
        if response1['DISEASE']=="":
            answer=input("what desease do you want to know about ?")
            response1['DISEASE']=answer
            return response1
        else:
            return response1

In [71]:
def IntentPrediction(question):
    ints =  predict_class(question,model)
    print(ints)
    res,tag = getResponse(ints, intents)
    print(res,tag)
    return res,tag
    

In [72]:
def IntentConfirmation(res1,tag):
    intent=""
    intent1=""
    for k,v in res1.items():
        if k != 'DISEASE':
            intent1=k
        if intent1=="" and tag!="":
            intent=tag
        if tag=="" and intent1!="":
            intent=intent1
        if tag!="" and intent1!="":
            intent=tag
        if tag=="" and intent1=="":
            question=input("do you whant to know the cause ,prevention,treatment or medicine fro this disease")
            intent=tag
        disease=res1['DISEASE']
        print(disease,intent)
        return disease,intent


In [80]:
question=input("enter your question")
res1=EntityDiseasePrediction(question)
res,tag=IntentPrediction(question)
disease,intent=IntentConfirmation(res1,tag)

enter your questionmy disease id cold
{'DISEASE': 'cold'}
1/1 [==============================] - 0s 10ms/step
[0.05248609 0.1607967  0.3567187  0.03878918 0.09273721 0.04267721
 0.19295897 0.06283598]
[{'intent': 'greeting', 'probability': '0.3567187'}, {'intent': 'thanks', 'probability': '0.19295897'}, {'intent': 'goodbye', 'probability': '0.1607967'}]
Hi, How is your health greeting
cold greeting
